# Setting Up

In [1]:
import torch
from torch import nn
import torchvision.datasets as Datasets
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

def image_transform(img):
    return T.ToTensor()(img)

mnist_train_ds = Datasets.MNIST(train=True, root=".", download=True, transform=image_transform)
mnist_test_ds = Datasets.MNIST(train=False, root=".", download=True, transform=image_transform)

mnist_train_dataloader = DataLoader(mnist_train_ds, batch_size=64, shuffle=True)
mnist_test_dataloader = DataLoader(mnist_test_ds, batch_size=64, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 87811954.14it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 40432474.57it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 22816892.54it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10904710.23it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [2]:
sample_imgs, sample_labels = next(iter(mnist_train_dataloader))
print(sample_imgs.shape)
px.imshow(sample_imgs[3].squeeze(dim=0))

torch.Size([64, 1, 28, 28])


# Model Defination

In [3]:
class CNN_Network(nn.Module):
    def __init__(self):
        super(CNN_Network, self).__init__()
        self.non_linearity=nn.ReLU
        self.kernel_size = 3
        self.padding_size=1
        self.max_pool_size=2
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 3, self.kernel_size, padding=self.padding_size ),
            self.non_linearity(),
            nn.Conv2d(3, 6, self.kernel_size, padding=self.padding_size ),
            self.non_linearity(),
            nn.MaxPool2d(self.max_pool_size),
            nn.Conv2d(6, 12, self.kernel_size, padding=self.padding_size ),
            self.non_linearity(),
            nn.Conv2d(12, 24, self.kernel_size, padding=self.padding_size ),
            self.non_linearity(),
            nn.MaxPool2d(self.max_pool_size),
            nn.Conv2d(24, 48, self.kernel_size, padding=self.padding_size ),
            self.non_linearity(),
            nn.MaxPool2d(self.max_pool_size)
        )
        self.linear_stack = nn.Sequential(
            nn.Linear(432, 10)
        )

    def forward(self, x):
        conv_output = self.conv_stack(x)
        linear_output = self.linear_stack(conv_output.flatten(start_dim=1))
        # print("Convolutional Output: ", conv_output.shape, conv_output.flatten(start_dim=1).shape)
        return linear_output
        # pass

In [5]:
test_net = CNN_Network()
test_net(sample_imgs).shape

torch.Size([64, 10])

# Training Loop Defination

In [6]:
def train(model, dataloader, loss_func, optimizer_func, verbose=True):
    model.train()
    loss_arr = []

    for batch, (X, y) in enumerate(dataloader):
        # X,y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_func(pred, y)
        optimizer_func.zero_grad()
        loss.backward()
        optimizer_func.step()

        loss_arr.append(loss.item())

        if(verbose):
            if(batch % 100==0):
                print(f'batch {batch} loss {loss.item()} [{batch*len(X)} / {len(dataloader.dataset)}]')
    return loss_arr

In [7]:
@torch.no_grad()
def test(model, dataloader):
    total = len(dataloader.dataset)
    correct = 0
    for batch, (X,y) in enumerate(dataloader):
        # X,y =  X.to(device), y.to(device)
        pred = model(X)
        correct += sum(pred.argmax(dim=1) == y)
    # print(f'Accuracy: {(correct/total)*100} {correct}/{total}')
    return [correct, total]


# Training Model

In [8]:
epochs = 11
learning_rate = 0.1

main_net = CNN_Network()

loss_func = nn.CrossEntropyLoss()
optimizer_func = torch.optim.SGD(main_net.parameters(), lr = learning_rate)

for i in range(epochs):
    print("Epoch: ", i+1)
    train(main_net, mnist_train_dataloader, loss_func, optimizer_func, verbose=False)
print("Done !")

Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Done !


# Validation

In [9]:
correct, total = test(main_net, mnist_train_dataloader)
print(f'Accuracy: {(correct/total)*100} [{correct}/{total}]')

Accuracy: 99.61000061035156 [59766/60000]


In [10]:
correct, total = test(main_net, mnist_test_dataloader)
print(f'Accuracy: {(correct/total)*100} [{correct}/{total}]')

Accuracy: 99.04999542236328 [9905/10000]
